## Vertex AI Gemini Fine-Tuning (PEFT) Quickstart

Based on the Feb 2025 Vertex AI documentation ("Tune Gemini models with adapters"), supervised
fine-tuning currently supports Gemini 1.0 Pro and the preview Gemini 1.5 adapter variants in
`us-central1`. This notebook follows that flow and replaces earlier cells that tried to fine-tune
unsupported model identifiers.

Before running the code cells:
- Enable billing for your Google Cloud project.
- Export `GOOGLE_CLOUD_PROJECT` (and optionally `GOOGLE_CLOUD_REGION`) or edit the constants in the
  configuration cell.
- Restart the runtime after any package upgrades.


In [1]:
import os
from datetime import datetime

try:
    from dotenv import load_dotenv
except ModuleNotFoundError:
    def load_dotenv():
        return None

import google.cloud.aiplatform as aiplatform
import vertexai

load_dotenv()

PROJECT_ID = os.environ.get('GOOGLE_CLOUD_PROJECT') or os.environ.get('PROJECT_ID') or 'periospot-mvp'
REGION = os.environ.get('GOOGLE_CLOUD_REGION', 'us-central1')

if PROJECT_ID in {'', 'your-project-id', '<project_id>'}:
    raise ValueError('Set PROJECT_ID or export GOOGLE_CLOUD_PROJECT before continuing.')

if REGION != 'us-central1':
    raise ValueError("Vertex AI Gemini supervised fine-tuning currently runs only in 'us-central1'.")

vertexai.init(project=PROJECT_ID, location=REGION)

print(f'Project: {PROJECT_ID}')
print(f'Region: {REGION}')
print(f'google-cloud-aiplatform: {aiplatform.__version__}')


Project: periospot-mvp
Region: us-central1
google-cloud-aiplatform: 1.119.0


In [2]:
from vertexai.generative_models import GenerativeModel
from vertexai.preview.tuning import sft
import json
from google.cloud import storage

SUPPORTED_BASE_MODELS = {
    'gemini-2.5-pro': 'GA',
    'gemini-2.5-flash': 'GA',
    'gemini-2.5-flash-lite': 'GA',
    'gemini-1.0-pro-002': 'GA',
    'gemini-1.5-pro-002': 'Preview',
    'gemini-1.5-flash-002': 'Preview',
    'text-bison@002': 'Legacy',
}

# ⚠️ THE GOOGLE SAMPLE DATASET HAS WRONG FORMAT!
# Old format: {"input_text": "...", "output_text": "..."}
# New format for Gemini 2.5: {"contents": [{"role": "user", "parts": [{"text": "..."}]}, ...]}

# Let's create a properly formatted training dataset
print("🔧 Creating properly formatted training dataset...")

# Sample training data in CORRECT format for Gemini 2.5
training_examples = [
    {
        "contents": [
            {"role": "user", "parts": [{"text": "What is machine learning?"}]},
            {"role": "model", "parts": [{"text": "Machine learning is a subset of artificial intelligence (AI) that enables computers to learn and improve from experience without being explicitly programmed. It uses algorithms and statistical models to analyze and draw inferences from patterns in data."}]}
        ]
    },
    {
        "contents": [
            {"role": "user", "parts": [{"text": "Explain neural networks in simple terms."}]},
            {"role": "model", "parts": [{"text": "Neural networks are computing systems inspired by the biological neural networks in animal brains. They consist of interconnected nodes (neurons) organized in layers that process information. These networks can learn to perform tasks by analyzing examples, gradually improving their accuracy through a process called training."}]}
        ]
    },
    {
        "contents": [
            {"role": "user", "parts": [{"text": "What is fine-tuning in AI?"}]},
            {"role": "model", "parts": [{"text": "Fine-tuning is a process where you take a pre-trained AI model and train it further on a specific dataset relevant to your task. This allows the model to adapt its knowledge to perform better on particular applications while retaining the general capabilities it learned during initial training."}]}
        ]
    },
]

# Save to a local file
import tempfile
import os

# Create temp directory for training data
temp_dir = tempfile.mkdtemp()
local_training_file = os.path.join(temp_dir, 'training_data.jsonl')

with open(local_training_file, 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

print(f"✅ Created training file: {local_training_file}")

# ⚠️ MUST upload to GCS - local files don't work!
print("\n📤 Uploading training data to GCS...")

# Create GCS bucket name (must be globally unique)
BUCKET_NAME = f'{PROJECT_ID}-gemini-tuning'
GCS_TRAINING_PATH = f'gs://{BUCKET_NAME}/training_data/training_{datetime.utcnow():%Y%m%d_%H%M%S}.jsonl'

try:
    # Initialize storage client
    storage_client = storage.Client(project=PROJECT_ID)
    
    # Create bucket if it doesn't exist
    try:
        bucket = storage_client.create_bucket(BUCKET_NAME, location='us-central1')
        print(f"✅ Created bucket: {BUCKET_NAME}")
    except Exception as e:
        if 'already exists' in str(e).lower() or '409' in str(e):
            bucket = storage_client.bucket(BUCKET_NAME)
            print(f"✅ Using existing bucket: {BUCKET_NAME}")
        else:
            raise
    
    # Upload the training file
    blob_name = GCS_TRAINING_PATH.replace(f'gs://{BUCKET_NAME}/', '')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(local_training_file)
    
    print(f"✅ Uploaded training data to: {GCS_TRAINING_PATH}")
    
    TRAINING_DATASET = GCS_TRAINING_PATH
    VALIDATION_DATASET = None
    
except Exception as e:
    print(f"❌ Error uploading to GCS: {e}")
    print("\n💡 ALTERNATIVE: Create your bucket manually:")
    print(f"   1. Go to https://console.cloud.google.com/storage")
    print(f"   2. Create bucket: {BUCKET_NAME}")
    print(f"   3. Upload {local_training_file} to the bucket")
    print(f"   4. Update TRAINING_DATASET = 'gs://{BUCKET_NAME}/your_file.jsonl'")
    raise

BASE_MODEL = 'gemini-2.5-pro'
TUNED_MODEL_DISPLAY_NAME = f'gemini-peft-{datetime.utcnow():%Y%m%d-%H%M%S}'

if BASE_MODEL not in SUPPORTED_BASE_MODELS:
    raise ValueError(f'Unsupported base model {BASE_MODEL}. Choose one of {list(SUPPORTED_BASE_MODELS)}.')

print(f'\n✅ Configuration:')
print(f'Base model: {BASE_MODEL} ({SUPPORTED_BASE_MODELS[BASE_MODEL]})')
print(f'Training dataset: {TRAINING_DATASET}')
print(f'Tuned model display name: {TUNED_MODEL_DISPLAY_NAME}')
print(f'\n🎉 Ready for fine-tuning with properly formatted dataset!')


🔧 Creating properly formatted training dataset...
✅ Created training file: /var/folders/cd/2c6x3jgj47j_fwl7231ts7m00000gn/T/tmpha9mjqre/training_data.jsonl

📤 Uploading training data to GCS...
✅ Using existing bucket: periospot-mvp-gemini-tuning
✅ Uploaded training data to: gs://periospot-mvp-gemini-tuning/training_data/training_20251007_152358.jsonl

✅ Configuration:
Base model: gemini-2.5-pro (GA)
Training dataset: gs://periospot-mvp-gemini-tuning/training_data/training_20251007_152358.jsonl
Tuned model display name: gemini-peft-20251007-152359

🎉 Ready for fine-tuning with properly formatted dataset!


In [ ]:
train_kwargs = {
    'source_model': BASE_MODEL,
    'train_dataset': TRAINING_DATASET,
    'tuned_model_display_name': TUNED_MODEL_DISPLAY_NAME
}
if VALIDATION_DATASET:
    train_kwargs['validation_dataset'] = VALIDATION_DATASET

print("🚀 Starting fine-tuning job...")
tuning_job = sft.train(**train_kwargs)

print(f'✅ Started tuning job: {tuning_job.resource_name}')
print(f'📊 Job name: {tuning_job.name}')
print(f'⏰ This may take 15-30 minutes to complete.')

# The job runs asynchronously - you can check status in the Cloud Console
# URL: https://console.cloud.google.com/vertex-ai/generative/language/tuning-jobs
print(f'\n🔗 Monitor progress at:')
print(f'   https://console.cloud.google.com/vertex-ai/locations/{REGION}/tuning-jobs/{tuning_job.name.split("/")[-1]}?project={PROJECT_ID}')

# For monitoring in the notebook, you can poll the status
print(f'\n⏳ Monitoring job status (this will wait until completion)...')
import time

# Poll every 30 seconds
max_wait_time = 3600  # 1 hour max
start_time = time.time()
check_interval = 30  # seconds

while True:
    # Refresh job status
    try:
        # Get current state
        current_state = tuning_job.state
        print(f"⏱️ Status: {current_state.name if hasattr(current_state, 'name') else current_state}")
        
        # Check if job is done
        if hasattr(current_state, 'name'):
            state_name = current_state.name
        else:
            state_name = str(current_state)
        
        if 'SUCCEEDED' in state_name or 'JOB_STATE_SUCCEEDED' in state_name:
            print("🎉 Fine-tuning completed successfully!")
            break
        elif 'FAILED' in state_name or 'CANCELLED' in state_name:
            print(f"❌ Fine-tuning failed with state: {state_name}")
            break
        
        # Check timeout
        elapsed = time.time() - start_time
        if elapsed > max_wait_time:
            print(f"⏰ Timeout after {max_wait_time/60:.1f} minutes")
            print("💡 Job is still running. Check Cloud Console for status.")
            break
        
        # Wait before next check
        time.sleep(check_interval)
        
    except Exception as e:
        print(f"⚠️ Error checking status: {e}")
        print("💡 Check Cloud Console for job status")
        break

# Get the tuned model information
try:
    if hasattr(tuning_job, 'tuned_model_name') and tuning_job.tuned_model_name:
        TUNED_MODEL_NAME = tuning_job.tuned_model_name
        print(f"\n✅ Tuned model name: {TUNED_MODEL_NAME}")
    
    if hasattr(tuning_job, 'tuned_model_endpoint_name') and tuning_job.tuned_model_endpoint_name:
        TUNED_MODEL_ENDPOINT = tuning_job.tuned_model_endpoint_name
        print(f"✅ Tuned model endpoint: {TUNED_MODEL_ENDPOINT}")
    else:
        print(f"\n⚠️ Model endpoint not yet available")
        print(f"💡 Check the Cloud Console URL above for the endpoint once training completes")
        
except Exception as e:
    print(f"⚠️ Could not retrieve tuned model info: {e}")
    print(f"💡 You can get the endpoint from Cloud Console once training completes")


🚀 Starting fine-tuning job...
Creating SupervisedTuningJob


E0000 00:00:1759850639.604962 16652295 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


SupervisedTuningJob created. Resource name: projects/1020593801304/locations/us-central1/tuningJobs/1542662865863835648
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/1020593801304/locations/us-central1/tuningJobs/1542662865863835648')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/1542662865863835648?project=1020593801304


✅ Started tuning job: projects/1020593801304/locations/us-central1/tuningJobs/1542662865863835648
📊 Job name: 1542662865863835648
⏰ This may take 15-30 minutes to complete.

🔗 Monitor progress at:
   https://console.cloud.google.com/vertex-ai/locations/us-central1/tuning-jobs/1542662865863835648?project=periospot-mvp

⏳ Monitoring job status (this will wait until completion)...
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING
⏱️ Status: JOB_STATE_PENDING


## 🎉 **SOLUTION**: Correctly Formatted Training Dataset

### ⚠️ **Root Cause of Error**
The error `"Row: 0. Missing required 'contents' field"` was caused by using Google's sample dataset which has an **OUTDATED FORMAT**.

### ✅ **Correct Format for Gemini 2.5**
Each training example must use this structure:
```json
{
  "contents": [
    {"role": "user", "parts": [{"text": "Your question here"}]},
    {"role": "model", "parts": [{"text": "Expected response here"}]}
  ]
}
```

The cell above creates a properly formatted dataset that will work with Gemini 2.5 fine-tuning!


In [ ]:
# 🔍 COMPREHENSIVE MODEL TESTING SYSTEM
# This cell tests which models work for inference and fine-tuning

import vertexai
from vertexai.generative_models import GenerativeModel
from vertexai.preview.tuning import sft
import time

# Initialize Vertex AI
PROJECT_ID = 'periospot-mvp'
REGION = 'europe-west1'
vertexai.init(project=PROJECT_ID, location=REGION)

# List of models to test - UPDATED with correct Gemini 2.5 models
MODELS_TO_TEST = [
    # Gemini 2.5 models (SUPPORTED for fine-tuning)
    'gemini-2.5-pro',
    'gemini-2.5-flash',
    'gemini-2.5-flash-lite',
    
    # Gemini 2.0 models (inference only)
    'gemini-2.0-flash',
    'gemini-2.0-flash-exp',
    'gemini-2.0-flash-thinking-exp',
    
    # Gemini 1.5 models (legacy, may not be available)
    'gemini-1.5-flash',
    'gemini-1.5-flash-8b',
    'gemini-1.5-pro',
    'gemini-1.5-pro-latest',
    
    # Gemini 1.0 models (legacy)
    'gemini-1.0-pro',
    'gemini-1.0-pro-latest',
    
    # Experimental models
    'gemini-exp-1206',
    'gemini-exp-1206-pro',
]

# Training dataset for fine-tuning tests
TRAINING_DATASET = 'gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl'

print("🚀 COMPREHENSIVE MODEL TESTING")
print("=" * 50)

# Results storage
inference_results = {}
fine_tuning_results = {}

for model_name in MODELS_TO_TEST:
    print(f"\n🔍 Testing model: {model_name}")
    print("-" * 40)
    
    # Test 1: Inference (generate_content)
    try:
        model = GenerativeModel(model_name)
        start_time = time.time()
        response = model.generate_content("What is a LLM?")
        end_time = time.time()
        
        inference_results[model_name] = {
            'status': '✅ SUCCESS',
            'response_time': f"{end_time - start_time:.2f}s",
            'response_preview': response.text[:100] + "...",
            'error': None
        }
        print(f"✅ Inference: SUCCESS ({end_time - start_time:.2f}s)")
        print(f"   Preview: {response.text[:80]}...")
        
    except Exception as e:
        inference_results[model_name] = {
            'status': '❌ FAILED',
            'response_time': None,
            'response_preview': None,
            'error': str(e)
        }
        print(f"❌ Inference: FAILED - {str(e)[:100]}...")
        continue  # Skip fine-tuning test if inference fails
    
    # Test 2: Fine-tuning (only if inference works)
    try:
        print(f"🔧 Testing fine-tuning...")
        tuning_job = sft.train(
            source_model=model_name,
            train_dataset=TRAINING_DATASET,
            tuned_model_display_name=f'test-tuning-{model_name.replace("-", "_")}',
        )
        
        fine_tuning_results[model_name] = {
            'status': '✅ SUPPORTED',
            'endpoint': tuning_job.tuned_model_endpoint_name,
            'error': None
        }
        print(f"✅ Fine-tuning: SUPPORTED")
        print(f"   Endpoint: {tuning_job.tuned_model_endpoint_name}")
        
    except Exception as e:
        fine_tuning_results[model_name] = {
            'status': '❌ NOT SUPPORTED',
            'endpoint': None,
            'error': str(e)
        }
        print(f"❌ Fine-tuning: NOT SUPPORTED - {str(e)[:100]}...")

# Summary Report
print("\n" + "=" * 60)
print("📊 TESTING SUMMARY REPORT")
print("=" * 60)

print("\n🔹 INFERENCE RESULTS:")
print("-" * 30)
for model, result in inference_results.items():
    status_emoji = "✅" if result['status'].startswith("✅") else "❌"
    print(f"{status_emoji} {model}: {result['status']}")

print("\n🔹 FINE-TUNING RESULTS:")
print("-" * 30)
for model, result in fine_tuning_results.items():
    status_emoji = "✅" if result['status'].startswith("✅") else "❌"
    print(f"{status_emoji} {model}: {result['status']}")

# Recommendations
print("\n🎯 RECOMMENDATIONS:")
print("-" * 20)
working_models = [m for m, r in inference_results.items() if r['status'].startswith("✅")]
fine_tuning_models = [m for m, r in fine_tuning_results.items() if r['status'].startswith("✅")]

if working_models:
    print(f"✅ Use these models for inference: {', '.join(working_models[:3])}")
else:
    print("❌ No models working for inference")

if fine_tuning_models:
    print(f"🔧 Use these models for fine-tuning: {', '.join(fine_tuning_models)}")
else:
    print("❌ No models support fine-tuning in this region/project")
    print("💡 Try using AI Studio API instead of Vertex AI for fine-tuning")

print(f"\n📈 Total tested: {len(MODELS_TO_TEST)} models")
print(f"✅ Inference working: {len(working_models)} models")
print(f"🔧 Fine-tuning supported: {len(fine_tuning_models)} models")


🚀 COMPREHENSIVE MODEL TESTING

🔍 Testing model: gemini-2.5-pro
----------------------------------------


E0000 00:00:1759642755.007235 1550705 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


✅ Inference: SUCCESS (27.16s)
   Preview: Of course! Here’s a detailed breakdown of what an LLM is, explained in simple te...
🔧 Testing fine-tuning...
Creating SupervisedTuningJob


E0000 00:00:1759642782.141396 1550705 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


❌ Fine-tuning: NOT SUPPORTED - 400 Row: 0. Missing required `contents` field....

🔍 Testing model: gemini-2.5-flash
----------------------------------------


...

E0000 00:00:1759642802.275764 1550705 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759642802.430768 1550705 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


❌ Inference: FAILED - 404 Publisher Model `projects/periospot-mvp/locations/europe-west1/publishers/google/models/gemini-1...

🔍 Testing model: gemini-1.0-pro-latest
----------------------------------------
❌ Inference: FAILED - 404 Publisher Model `projects/periospot-mvp/locations/europe-west1/publishers/google/models/gemini-1...

🔍 Testing model: gemini-exp-1206
----------------------------------------


E0000 00:00:1759642802.554591 1550705 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759642802.707370 1550705 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


❌ Inference: FAILED - 404 Publisher Model `projects/periospot-mvp/locations/europe-west1/publishers/google/models/gemini-e...

🔍 Testing model: gemini-exp-1206-pro
----------------------------------------
❌ Inference: FAILED - 404 Publisher Model `projects/periospot-mvp/locations/europe-west1/publishers/google/models/gemini-e...

📊 TESTING SUMMARY REPORT

🔹 INFERENCE RESULTS:
------------------------------
✅ gemini-2.5-pro: ✅ SUCCESS
✅ gemini-2.5-flash: ✅ SUCCESS
✅ gemini-2.5-flash-lite: ✅ SUCCESS
✅ gemini-2.0-flash: ✅ SUCCESS
❌ gemini-2.0-flash-exp: ❌ FAILED
❌ gemini-2.0-flash-thinking-exp: ❌ FAILED
❌ gemini-1.5-flash: ❌ FAILED
❌ gemini-1.5-flash-8b: ❌ FAILED
❌ gemini-1.5-pro: ❌ FAILED
❌ gemini-1.5-pro-latest: ❌ FAILED
❌ gemini-1.0-pro: ❌ FAILED
❌ gemini-1.0-pro-latest: ❌ FAILED
❌ gemini-exp-1206: ❌ FAILED
❌ gemini-exp-1206-pro: ❌ FAILED

🔹 FINE-TUNING RESULTS:
------------------------------
❌ gemini-2.5-pro: ❌ NOT SUPPORTED
❌ gemini-2.5-flash: ❌ NOT SUPPORTED
❌ gemini-2.5-flash-lite:

E0000 00:00:1759642802.851959 1550705 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [ ]:
# 🧪 Test the tuned model (run this after fine-tuning completes)

# Check if we have the tuned model endpoint
if 'TUNED_MODEL_ENDPOINT' in globals() and TUNED_MODEL_ENDPOINT:
    print(f"🧪 Testing tuned model: {TUNED_MODEL_ENDPOINT}")
    
    try:
        tuned_model = GenerativeModel(TUNED_MODEL_ENDPOINT)
        
        # Test questions
        test_questions = [
            'What is a large language model?',
            'What is machine learning?',
            'Explain fine-tuning in AI.'
        ]
        
        for i, question in enumerate(test_questions, 1):
            print(f"\n❓ Question {i}: {question}")
            response = tuned_model.generate_content(question)
            print(f"💬 Answer: {response.text}")
            print("-" * 50)
        
        print("\n✅ Tuned model is working perfectly!")
        
    except Exception as e:
        print(f"❌ Error testing tuned model: {e}")
        print(f"💡 Make sure fine-tuning completed successfully")
        
elif 'TUNED_MODEL_NAME' in globals() and TUNED_MODEL_NAME:
    print(f"🧪 Testing tuned model: {TUNED_MODEL_NAME}")
    
    try:
        tuned_model = GenerativeModel(TUNED_MODEL_NAME)
        response = tuned_model.generate_content('What is a large language model?')
        print(f"💬 Response: {response.text}")
        print("\n✅ Tuned model is working!")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        
else:
    print("⚠️ No tuned model endpoint found")
    print("💡 Steps:")
    print("   1. Make sure fine-tuning cell completed successfully")
    print("   2. Check Cloud Console for the tuned model endpoint")
    print("   3. Set: TUNED_MODEL_ENDPOINT = 'your-endpoint-here'")
    print("   4. Re-run this cell")
    
    # You can manually set the endpoint here:
    # TUNED_MODEL_ENDPOINT = "projects/.../locations/.../endpoints/..."
    # tuned_model = GenerativeModel(TUNED_MODEL_ENDPOINT)
    # response = tuned_model.generate_content('What is a large language model?')
    # print(response.text)


Snippet 2. Using Vertex AI and Google AI studio SDKs for unimodal text generation 

In [ ]:
%pip install --upgrade --quiet google-genai

In [ ]:
import sys 
if "google.colab" in sys.modules:
    from google.colab import auth  
    auth.authenticate_user() 
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    MediaResolution,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    ToolCodeExecution,
    VertexAISearch,
)
import os

load_dotenv()


PROJECT_ID = os.getenv('PROJECT_ID')  # @param {type: “string”, placeholder: “[your-project-id]”, isTemplate: true}
if not PROJECT_ID or PROJECT_ID == PROJECT_ID:
    PROJECT_ID = str(os.environ.get(PROJECT_ID)) 
LOCATION = os.environ.get(REGION)  
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION) 
MODEL_ID = "gemini-2.0-flash-001" # @param {type: “string”} 
response = client.models.generate_content(
    model=MODEL_ID, contents="What’s the largest planet in our solar system?"
)
display(Markdown(response.text))